In [15]:
import os
import sys
# SRC_DIR environment variable should be the absolute path to the 'multicopy-STR-genotyping' directory
sys.path.append(os.environ["SRC_DIR"])

In [43]:
import gzip

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pysam
import seaborn as sns

from multicopy_STR_genotyping import str_utils

sns.set_context("poster")
%matplotlib inline

## Investigating loci that were wrongly called from simulated sequencing reads

We used wgsim to simulate short sequencing reads to model trisomy 21. For this experiment, we combined reads from chr21 in GRCh38 and the paternal & maternal haplotypes of HG002, which we then mapped to GRCh38. For ~1% of STR loci that we genotype in this sample, the genotype ConSTRain estimates does not match the genotypes in the input haplotypes ([2024-05-23_tri21_simulated.ipynb](2024-05-23_tri21_simulated.ipynb)). Here, we investigate whether this is because there were simply not many reads simulated for this region, or because the reads for a specific allele length get spuriously mapped to other parts of the chromosome.

We use chr21_44468366 as an example. This is a tetranucleotide repeat with simulated allele lengths [8, 11, 12] (11 in GRCh38). The genotype we estimated with ConSTRain, however, was [8, 12, 12], which was estimated from allele frequency distribution {8: 7, 11: 4, 12: 14}.

In [81]:
# chr21:44468366-44468409
str_id = "chr21_44468366"
chr = "chr21"
str_start = 44468366 # 1-based coordinates
str_end = 44468409

file_fw = "../../data/simulated_reads/reads/GRCh38_chr21_1.fq.gz"
file_rv = "../../data/simulated_reads/reads/GRCh38_chr21_2.fq.gz"

Wgsim includes the source reference coordinates in the names of fastq records. We use this to determine which inserts should *in principle* overlap the STR locus of interest. The coordinates in fastq names specify the start and end position of the insert that was generated. To figure out if either of the two reads simulated for this insert we check whether the STR locus is in the first or the last 150 positions of the insert

In [96]:
def get_overlapping_reads(fastq, str_start, str_end, remove=None):
    read_ids = []
    with gzip.open(fastq, 'rt') as f:
        for line in f:
            if line.startswith("@"):
                split = line.split("_")
                # read_start, read_end = int(split[3]), int(split[4])
                read_start, read_end = int(split[1]), int(split[2])
                fw_overlap = str_utils.range_overlap(read_start, read_start + 150, str_start, str_end)
                rv_overlap = str_utils.range_overlap(read_end - 150, read_end, str_start, str_end)
                str_len = str_end - str_start + 1
                if fw_overlap == str_len or rv_overlap == str_len:
                    hit = line.strip()
                    if remove:
                        for i in remove:
                            hit = hit.replace(i, "")
                    read_ids.append(hit)                    
    return set(read_ids)

In [97]:
reads_fw = get_overlapping_reads(file_fw, str_start, str_end, remove = ["@", "/1"])
len(reads_fw)

4

In [98]:
for i in reads_fw:
    print(i)

chr21_44468274_44468818_0:0:0_0:0:0_232d2f
chr21_44467970_44468452_0:0:0_0:0:0_9b5fc
chr21_44467968_44468503_0:0:0_0:0:0_13ea48
chr21_44467874_44468451_0:0:0_0:0:0_12036d


There are only four reads simulated from GRCh38 that overlap the chr21:44468366-44468409 STR locus. This is the same number of reads that ConSTRain observed for the GRCh38 allele length at this locus (11). This means that the wrong call stemmed from the fact that this locus contributed fewer reads to the distribution due to the randomness in how wgsim generates reads.

In [100]:
alignment = pysam.AlignmentFile("../../data/simulated_reads/alignments/GRCh38_chr21_GRCh38.bam", 'rb')

for record in alignment.fetch():
    if record.query_name in reads_fw:
        split = record.query_name.split("_")
        # read_start, read_end = int(split[3]), int(split[4])
        read_start, read_end = int(split[1]), int(split[2])
        if record.reference_start + 1 < read_start or record.reference_start + 151 > read_end:
            print(record.query_name, record.reference_start)
        # break

alignment.close()

chr21_44467874_44468451_0:0:0_0:0:0_12036d 44468301
chr21_44467970_44468452_0:0:0_0:0:0_9b5fc 44468302
chr21_44467968_44468503_0:0:0_0:0:0_13ea48 44468353
chr21_44468274_44468818_0:0:0_0:0:0_232d2f 44468668
